# Neptune ML Models - Model Registry

This notebook trains ML models for the Neptune Intelligence Agent:
- **Consumption Forecasting** - Predict future monthly water consumption
- **Utility Churn Prediction** - Classify utilities at risk of churning
- **Meter Deployment Success** - Predict which meter deployments will be successful

All models are registered to Snowflake Model Registry.


## Import Required Packages


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_absolute_error, accuracy_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake


In [ ]:
session = get_active_session()
session.use_database('NEPTUNE_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('NEPTUNE_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")


---
# MODEL 1: Consumption Forecasting

Predict future monthly water consumption using historical data.


### Prepare Consumption Training Data


In [ ]:
consumption_df = session.sql("""
SELECT
    DATE_TRUNC('month', reading_timestamp)::DATE AS reading_month,
    MONTH(reading_timestamp) AS month_num,
    YEAR(reading_timestamp) AS year_num,
    SUM(consumption_gallons)::FLOAT AS total_consumption,
    COUNT(DISTINCT meter_serial_number)::FLOAT AS active_meters,
    COUNT(DISTINCT utility_id)::FLOAT AS active_utilities
FROM RAW.METER_READINGS
WHERE reading_timestamp >= DATEADD('month', -36, CURRENT_DATE())
GROUP BY 1, 2, 3
ORDER BY 1
""")

print(f"Consumption data: {consumption_df.count()} months")
consumption_df.show(5)


### Train Consumption Model


In [ ]:
train_consumption, test_consumption = consumption_df.random_split([0.8, 0.2], seed=42)

train_consumption = train_consumption.drop("READING_MONTH")
test_consumption = test_consumption.drop("READING_MONTH")

consumption_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["MONTH_NUM", "ACTIVE_METERS", "ACTIVE_UTILITIES"],
        output_cols=["MONTH_NUM_SCALED", "ACTIVE_METERS_SCALED", "ACTIVE_UTILITIES_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["TOTAL_CONSUMPTION"],
        output_cols=["PREDICTED_CONSUMPTION"]
    ))
])

consumption_pipeline.fit(train_consumption)
print("✅ Consumption forecasting model trained")


### Evaluate and Register Consumption Model


In [ ]:
predictions = consumption_pipeline.predict(test_consumption)
mae = mean_absolute_error(df=predictions, y_true_col_names="TOTAL_CONSUMPTION", y_pred_col_names="PREDICTED_CONSUMPTION")
metrics = {"mae": round(mae, 2)}
print(f"Consumption model metrics: {metrics}")

reg = Registry(session)
reg.log_model(
    model=consumption_pipeline,
    model_name="CONSUMPTION_FORECASTER",
    version_name="V1",
    comment="Predicts monthly water consumption using Linear Regression.",
    metrics=metrics
)

print("✅ Consumption model registered to Model Registry as CONSUMPTION_FORECASTER")


---
# MODEL 2: Utility Churn Prediction

Classify utilities as likely to churn or not based on their operational patterns.


### Prepare Churn Training Data


In [ ]:
churn_df = session.sql("""
SELECT
    u.utility_id,
    u.utility_type,
    u.service_population::FLOAT AS service_population,
    COUNT(DISTINCT wo.work_order_id)::FLOAT AS total_work_orders,
    AVG(wo.customer_satisfaction_score)::FLOAT AS avg_satisfaction,
    COUNT(DISTINCT CASE WHEN mr.anomaly_detected THEN mr.reading_id END)::FLOAT AS anomaly_count,
    (u.utility_status = 'INACTIVE')::BOOLEAN AS is_churned -- Simplified target
FROM RAW.UTILITIES u
LEFT JOIN RAW.WORK_ORDERS wo ON u.utility_id = wo.utility_id
LEFT JOIN RAW.METER_READINGS mr ON u.utility_id = mr.utility_id
WHERE u.utility_status IN ('ACTIVE', 'INACTIVE')
GROUP BY 1, 2, 3, 7
LIMIT 5000
""")

print(f"Churn data: {churn_df.count()} utilities")
churn_df.show(5)


### Train Churn Classification Model


In [ ]:
train_churn, test_churn = churn_df.random_split([0.8, 0.2], seed=42)

train_churn = train_churn.drop("UTILITY_ID")
test_churn = test_churn.drop("UTILITY_ID")

churn_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["UTILITY_TYPE"],
        output_cols=["UTILITY_TYPE_ENCODED"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_CHURNED"],
        output_cols=["CHURN_PREDICTION"],
        n_estimators=100
    ))
])

churn_pipeline.fit(train_churn)
print("✅ Churn classification model trained")


### Evaluate and Register Churn Model


In [ ]:
churn_predictions = churn_pipeline.predict(test_churn)
accuracy = accuracy_score(df=churn_predictions, y_true_col_names="IS_CHURNED", y_pred_col_names="CHURN_PREDICTION")
churn_metrics = {"accuracy": round(accuracy, 4)}
print(f"Churn model metrics: {churn_metrics}")

reg.log_model(
    model=churn_pipeline,
    model_name="UTILITY_CHURN_PREDICTOR",
    version_name="V1",
    comment="Predicts utility churn risk using a Random Forest classifier.",
    metrics=churn_metrics
)

print("✅ Churn model registered to Model Registry as UTILITY_CHURN_PREDICTOR")


---
# MODEL 3: Meter Deployment Success Prediction

Predict which meter deployments will be successful (i.e., remain active and report data).


### Prepare Deployment Success Data


In [ ]:
success_df = session.sql("""
SELECT
    mi.meter_serial_number,
    mc.meter_family,
    mc.technology,
    u.utility_type,
    DATEDIFF('year', u.onboarding_date, CURRENT_DATE())::FLOAT AS utility_age_years,
    (mi.meter_status = 'DEPLOYED' AND mi.last_sync_date >= DATEADD('month', -3, CURRENT_DATE()))::BOOLEAN AS is_successful
FROM RAW.METER_INVENTORY mi
JOIN RAW.METER_CATALOG mc ON mi.meter_model_id = mc.meter_model_id
JOIN RAW.UTILITIES u ON mi.utility_id = u.utility_id
LIMIT 10000 -- Limit for faster training
""")

print(f"Deployment success data: {success_df.count()} meters")
success_df.show(5)


### Train Deployment Success Model


In [ ]:
train_success, test_success = success_df.random_split([0.8, 0.2], seed=42)

train_success = train_success.drop("METER_SERIAL_NUMBER")
test_success = test_success.drop("METER_SERIAL_NUMBER")

success_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["METER_FAMILY", "TECHNOLOGY", "UTILITY_TYPE"],
        output_cols=["METER_FAMILY_ENC", "TECHNOLOGY_ENC", "UTILITY_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", LogisticRegression(
        label_cols=["IS_SUCCESSFUL"],
        output_cols=["SUCCESS_PREDICTION"]
    ))
])

success_pipeline.fit(train_success)
print("✅ Deployment success model trained")


### Evaluate and Register Deployment Success Model


In [ ]:
success_predictions = success_pipeline.predict(test_success)
accuracy = accuracy_score(df=success_predictions, y_true_col_names="IS_SUCCESSFUL", y_pred_col_names="SUCCESS_PREDICTION")
success_metrics = {"accuracy": round(accuracy, 4)}
print(f"Deployment success model metrics: {success_metrics}")

reg.log_model(
    model=success_pipeline,
    model_name="DEPLOYMENT_SUCCESS_PREDICTOR",
    version_name="V1",
    comment="Predicts meter deployment success using Logistic Regression.",
    metrics=success_metrics
)

print("✅ Deployment success model registered as DEPLOYMENT_SUCCESS_PREDICTOR")


---
# Verify Models in Registry


In [ ]:
print("Models in registry:")
reg.show_models()
